<a href="https://colab.research.google.com/github/varmams2023/phd/blob/main/2WorkingExtensionWithPCGANBlockChainWithChequesFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install colorama  # Install the colorama library

import hashlib
import json
import pandas as pd
from time import time
from typing import List
from colorama import Fore, Style, init
from tqdm import tqdm  # For progress bar
from google.colab import drive
import random  # For simulating the PCGAN score

# ---- Section 1: Install & Initialize Libraries ----
# Mount Google Drive to access the CSV file
drive.mount('/content/drive')

# Initialize colorama for colored outputs
init(autoreset=True)

# ---- Section 2: Block Class ----

class Block:
    """Class to represent each block in the blockchain."""

    def __init__(self, index, timestamp, cheque_data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.cheque_data = cheque_data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        """Calculate the hash of the current block."""
        block_string = json.dumps(self.__dict__, sort_keys=True)
        return hashlib.sha256(block_string.encode()).hexdigest()

# Debug: Block Class Initialized
print("Block Class Initialized.")

# ---- Section 3: Blockchain Class ----

class Blockchain:
    """Class to handle blockchain operations."""

    def __init__(self):
        self.chain = []
        self.pending_cheques = []
        self.create_genesis_block()

    def create_genesis_block(self):
        """Create the first block (genesis block)."""
        genesis_block = Block(0, time(), {"check_number": "0", "payee_name": "Genesis", "bank_name": "Genesis Bank", "account_number": "000000", "ifsc_code": "000000"}, "0")
        self.chain.append(genesis_block)

    def add_cheque(self, cheque_data):
        """Add cheque data as a new block to the blockchain."""
        previous_hash = self.chain[-1].hash
        new_block = Block(len(self.chain), time(), cheque_data, previous_hash)
        self.chain.append(new_block)
        self.pending_cheques = []  # Reset pending cheques after adding

    def add_cheques_to_blockchain(self, cheques: List[dict]):
        """Method to add multiple cheques to the blockchain."""
        for cheque in tqdm(cheques, desc="Adding Cheques to Blockchain", unit="cheque"):
            self.add_cheque(cheque)

    def validate_cheque(self, cheque_data):
        """Validate if the given cheque data exists in the blockchain."""
        for block in self.chain:
            if block.cheque_data == cheque_data:
                return True  # Valid cheque data found
        return False

    def display_chain(self, num_blocks=15):
        """Display a limited number of blocks from the blockchain."""
        print(f"\nDisplaying first {num_blocks} blocks from the blockchain:\n")
        for block in self.chain[:num_blocks]:
            print(f"\n{Fore.GREEN}Block #{block.index} ({Fore.CYAN}{block.timestamp}{Style.RESET_ALL})")
            print(f"{Fore.YELLOW}Check Number: {block.cheque_data['check_number']}")
            print(f"{Fore.YELLOW}Payee Name: {block.cheque_data['payee_name']}")
            print(f"{Fore.YELLOW}Bank Name: {block.cheque_data['bank_name']}")
            print(f"{Fore.YELLOW}Account Number: {block.cheque_data['account_number']}")
            print(f"{Fore.YELLOW}IFSC Code: {block.cheque_data['ifsc_code']}")
            print(f"{Fore.BLUE}Hash: {block.hash}\n")

    def count_blocks_by_bank(self):
        """Count the number of blocks grouped by bank."""
        bank_count = {}
        for block in self.chain:
            bank_name = block.cheque_data['bank_name']
            if bank_name in bank_count:
                bank_count[bank_name] += 1
            else:
                bank_count[bank_name] = 1
        return bank_count

    def total_blocks(self):
        """Return the total number of blocks in the blockchain."""
        return len(self.chain)

# ---- Section 4: Loading Cheque Data from CSV ----

def load_cheque_data_from_csv(file_path: str):
    """Load cheque data from a CSV file."""
    df = pd.read_csv(file_path)

    # Ensure only relevant columns are selected
    cheque_data = df[['CheckNumber', 'AccountNumber', 'PayeeName', 'BankName', 'IFSCCode']]

    # Convert the DataFrame into a list of dictionaries
    cheque_data_list = cheque_data.rename(columns={
        'CheckNumber': 'check_number',
        'AccountNumber': 'account_number',
        'PayeeName': 'payee_name',
        'BankName': 'bank_name',
        'IFSCCode': 'ifsc_code'
    }).to_dict(orient='records')

    return cheque_data_list

# File path for the CSV in Google Drive
file_path = '/content/drive/MyDrive/phd/dataset/ChequeDataSetFull.csv'

# Load cheque data from CSV
cheque_data_list = load_cheque_data_from_csv(file_path)

# Debug: Data Loaded from CSV
print(f"Loaded {len(cheque_data_list)} cheque records from CSV.")

# ---- Section 5: Initialize Blockchain ----

# Initialize the blockchain
blockchain = Blockchain()

# Add cheques to the blockchain
blockchain.add_cheques_to_blockchain(cheque_data_list)

# Debug: Blockchain Initialized and Cheques Added
print(f"Blockchain initialized with {len(blockchain.chain)} blocks.")

# ---- Section 6: Simulated PCGAN Score ----

def generate_pcg_score():
    """Simulate a Progressive Conditional Generative Adversarial Network (PCGAN) score."""
    # Generate a random score between 0 (fake) and 1 (valid)
    return round(random.uniform(0, 1), 2)

# ---- Section 7: Menu for User Input ----

def display_menu():
    """Display the menu and handle user input."""
    while True:
        print("\nMenu:")
        print("1. Print Blockchain")
        print("2. Add Block (Manually input cheque data)")
        print("3. Validate if Cheque is present")
        print("4. Print Count of Blocks by Bank")
        print("5. Exit")
        choice = input("Enter your choice (1/2/3/4/5): ")

        if choice == '1':
            # Take number of blocks to print as input
            num_blocks = int(input("Enter the number of blocks to display: "))
            blockchain.display_chain(num_blocks=num_blocks)

        elif choice == '2':
            # Manually input the fields for a new block
            check_number = input("Enter Check Number: ")
            payee_name = input("Enter Payee Name: ")
            bank_name = input("Enter Bank Name: ")
            account_number = input("Enter Account Number: ")
            ifsc_code = input("Enter IFSC Code: ")

            cheque_data = {
                'check_number': check_number,
                'payee_name': payee_name,
                'bank_name': bank_name,
                'account_number': account_number,
                'ifsc_code': ifsc_code
            }
            blockchain.add_cheque(cheque_data)
            print(Fore.GREEN + "Block added successfully!")

        elif choice == '3':
            # Manually input the fields for cheque validation
            check_number = input("Enter Check Number: ")
            payee_name = input("Enter Payee Name: ")
            bank_name = input("Enter Bank Name: ")
            account_number = input("Enter Account Number: ")
            ifsc_code = input("Enter IFSC Code: ")

            cheque_data = {
                'check_number': check_number,
                'payee_name': payee_name,
                'bank_name': bank_name,
                'account_number': account_number,
                'ifsc_code': ifsc_code
            }
            is_valid = blockchain.validate_cheque(cheque_data)
            if is_valid:
                print(Fore.GREEN + "Valid Cheque")
            else:
                # Simulate PCGAN Score for the cheque
                pcgan_score = generate_pcg_score()
                result = "Fake" if pcgan_score < 0.5 else "Valid"
                print(Fore.RED + f"Cheque not present in blockchain. PCGAN Score: {pcgan_score} ({result} Cheque)")

        elif choice == '4':
            # Print count of blocks grouped by bank and total block count
            bank_count = blockchain.count_blocks_by_bank()
            total_blocks = blockchain.total_blocks()
            print(f"\nTotal Blocks in Blockchain: {total_blocks}")
            print("\nCount of Blocks by Bank:")
            for bank, count in bank_count.items():
                print(f"{Fore.YELLOW}{bank}: {count} blocks")

        elif choice == '5':
            print("Exiting...")
            break
        else:
            print(Fore.RED + "Invalid choice. Please try again.")

# Debug: Menu Initialized
print("Menu system initialized.")

# Start the menu
display_menu()


Mounted at /content/drive
Block Class Initialized.
Loaded 100000 cheque records from CSV.


Adding Cheques to Blockchain: 100%|██████████| 100000/100000 [00:02<00:00, 34026.47cheque/s]


Blockchain initialized with 100001 blocks.
Menu system initialized.

Menu:
1. Print Blockchain
2. Add Block (Manually input cheque data)
3. Validate if Cheque is present
4. Print Count of Blocks by Bank
5. Exit
Enter your choice (1/2/3/4/5): 3
Enter Check Number: 234234
Enter Payee Name: 234234
Enter Bank Name: 234234
Enter Account Number: 234234
Enter IFSC Code: 234234
Cheque not present in blockchain. PCGAN Score: 0.4 (Fake Cheque)

Menu:
1. Print Blockchain
2. Add Block (Manually input cheque data)
3. Validate if Cheque is present
4. Print Count of Blocks by Bank
5. Exit
